- Object Classification


In [ ]:
# On a utiliser python 3.7

# Installer Keras
!python -m pip install Keras==2.2.4 Keras-Applications==1.0.8 Keras-Preprocessing==1.1.2

# Installer TensorFlow
!python -m pip install tensorflow==1.15.0 tensorflow-estimator==1.15.1 tensorboard==1.15.0

# Installer Scikit-Image (en installant également ses dépendances courantes)
!python -m pip install scikit-image==0.16.2

# Autres dépendances essentielles
!python -m pip install numpy==1.21.6 scipy==1.7.3 pandas==1.0.3 matplotlib==3.5.3 Pillow==9.5.0

# Packages supplémentaires pour la manipulation et l'affichage des données
!python -m pip install seaborn==0.11.2 tqdm==4.66.6 requests==2.31.0

# Pour la manipulation JSON et les fichiers de labels (Labelme2COCO, Pybboxes)
!python -m pip install jsonschema==4.17.3 labelme2coco==0.2.6 pybboxes==0.1.6

# Installations pour des modules auxiliaires comme l'interface utilisateur et le client Jupyter
!python -m pip install jupyter-client==7.4.9 jupyter-core==4.12.0 nest-asyncio==1.6.0

# Packages pour le développement d'algorithmes et calculs mathématiques avancés
!python -m pip install absl-py==2.1.0 cloudpickle==2.2.1 dask==2022.2.0

# Packages de manipulation d'images et de fichiers multimédia
!python -m pip install opencv-python==3.4.13.47 imageio==2.10.4

# Installations pour les réseaux de neurones et autres applications IA
!python -m pip install torch==1.13.1 sahi==0.11.18

# Installations pour gérer les fichiers et le système
!python -m pip install fsspec==2023.1.0 importlib-resources==5.12.0 importlib-metadata==6.7.0

# Installations pour la manipulation de JSON et la gestion des chemins de fichiers
!python -m pip install attrs==24.2.0 pyparsing==3.1.4 typing-extensions==4.7.1

# Gestion des demandes réseau et de l'interface utilisateur
!python -m pip install urllib3==1.26.16 termcolor==2.3.0 colorama==0.4.6

# Autres utilitaires importants pour le système et les structures de données
!python -m pip install setuptools==68.0.0 wheel==0.42.0 wrapt==1.16.0

- Prépapration des données 

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

def preprocess_annotations(dataset_dir, subset):
    """
    Corrige les annotations pour que tous les polygones soient valides :
    - Limite les coordonnées des points des polygones aux dimensions de l'image.
    - Supprime les polygones invalides (moins de 3 points).

    Args:
        dataset_dir (str): Répertoire contenant les données.
        subset (str): Sous-ensemble à traiter ("train" ou "val").
    """
    # Répertoire du sous-ensemble (train ou val)
    subset_dir = os.path.join(dataset_dir, subset)
    
    # Lister et recuperer tous les fichiers JSON dans le répertoire
    json_files = [f for f in os.listdir(subset_dir) if f.endswith(".json")]

    # Parcourir chaque fichier JSON
    for json_file in json_files:
        json_path = os.path.join(subset_dir, json_file)  # Chemin complet du fichier JSON

        # Charger les annotations du fichier
        with open(json_path, 'r') as f:
            annotations = json.load(f)

        # Charger l'image associée pour obtenir ses dimensions
        image_path = os.path.join(subset_dir, annotations['imagePath'])
        image = plt.imread(image_path)  # Charger l'image
        height, width = image.shape[:2]  # Récupérer la hauteur et la largeur de l'image

        # Liste pour stocker les formes valides
        valid_shapes = []

        # Parcourir chaque polygone dans les annotations
        for shape in annotations['shapes']:
            points = shape['points']  # Points du polygone
            corrected_points = []

            # Corriger chaque point du polygone
            for x, y in points:
                # Limiter les coordonnées à l'intérieur des dimensions de l'image
                corrected_x = max(0, min(width - 1, x))  # x dans [0, width-1]
                corrected_y = max(0, min(height - 1, y))  # y dans [0, height-1]
                corrected_points.append([corrected_x, corrected_y])

            # Vérifier si le polygone corrigé est valide (au moins 3 points)
            if len(corrected_points) >= 3:
                shape['points'] = corrected_points  # Mettre à jour les points corrigés
                valid_shapes.append(shape)  # Ajouter la forme corrigée à la liste des formes valides

        # Mettre à jour les annotations avec uniquement les polygones valides
        annotations['shapes'] = valid_shapes

        # Sauvegarder les annotations corrigées dans le fichier JSON
        with open(json_path, 'w') as f:
            json.dump(annotations, f, indent=4)  # Sauvegarde avec un format JSON lisible

# Appliquer le prétraitement sur les ensembles d'entraînement et de validation
dataset_dir = 'data_t'
preprocess_annotations(dataset_dir, 'train')  # Prétraiter les annotations d'entraînement
preprocess_annotations(dataset_dir, 'val')  # Prétraiter les annotations de validation


- entrainement du modele

In [ ]:
import os
import json
import numpy as np
import imgaug
import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.model import MaskRCNN
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import logging

# Configuration du logger pour enregistrer les erreurs dans un fichier log
logging.basicConfig(filename="training_errors.log", level=logging.ERROR)

# Configuration personnalisée pour Mask R-CNN
class FlowerConfig(Config):
    NAME = "flower_tulip"  # Nom du modele 
    IMAGES_PER_GPU = 3  # Nombre d'images par GPU pendant l'entraînement
    NUM_CLASSES = 1 + 2  # Fond(background) + 2 classes : Tulipe et Non-Tulipe
    STEPS_PER_EPOCH = 53  # Nombre de pas (batches) par époque
    DETECTION_MIN_CONFIDENCE = 0.8  # Seuil minimum pour considérer une détection valide

# Définition d'une classe pour charger et préparer le dataset
class FlowerDataset(utils.Dataset):
    def load_flowers(self, dataset_dir, subset):
        """
        Charger les annotations et images pour un sous-ensemble (train ou val).
        """
        # Ajouter les classes au dataset
        self.add_class("flower", 1, "tulipe")  # Classe pour les tulipes
        self.add_class("flower", 2, "non_tulipe")  # Classe pour les autres fleurs

        # Vérifier que le sous-ensemble est valide
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Charger les annotations (format JSON) pour chaque fichier
        for filename in os.listdir(dataset_dir):
            if filename.endswith(".json"):
                json_path = os.path.join(dataset_dir, filename)
                with open(json_path) as f:
                    annotations = json.load(f)
                    polygons = [shape['points'] for shape in annotations['shapes']]  # Points des polygones
                    label_name = annotations['shapes'][0]['label']  # Classe associée à l'objet
                    label_id = 1 if label_name == "tulipe" else 2  # Mappe "tulipe" à 1 et "non_tulipe" à 2
                    
                    # Trouver le chemin de l'image associée (supporte plusieurs extensions)
                    image_name = annotations['imagePath']
                    image_path = None
                    for ext in ['.png', '.jpg', '.jpeg']:
                        potential_path = os.path.join(dataset_dir, os.path.splitext(image_name)[0] + ext)
                        if os.path.exists(potential_path):
                            image_path = potential_path
                            break
                    
                    # Si l'image n'est pas trouvée, ignorer l'annotation
                    if image_path is None:
                        print(f"Image {image_name} non trouvée pour le fichier {json_path}")
                        continue
                    
                    # Charger les dimensions de l'image
                    image = plt.imread(image_path)
                    height, width = image.shape[:2]

                    # Ajouter l'image et ses métadonnées au dataset
                    self.add_image(
                        "flower",  # Nom de la source
                        image_id=filename,  # Identifiant unique
                        path=image_path,
                        width=width, height=height,
                        polygons=polygons,
                        class_id=label_id  # Classe associée
                    )

    def load_mask(self, image_id):
        """
        Charger les masques pour une image donnée.
        """
        # Récupérer les informations sur l'image
        image_info = self.image_info[image_id]
        if image_info["source"] != "flower":
            return super(self.__class__, self).load_mask(image_id)

        # Initialiser un masque vide
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])], dtype=np.uint8)
        class_ids = np.array([info["class_id"]] * len(info["polygons"]))  # ID de la classe pour chaque masque

        # Remplir le masque pour chaque polygone
        for i, polygon in enumerate(info["polygons"]):
            polygon = np.array(polygon, dtype=int)  # Convertir en tableau NumPy (np)
            rr, cc = polygon[:, 1].astype(int), polygon[:, 0].astype(int)  # Extraire les coordonnées
            mask[rr, cc, i] = 1

        return mask.astype(bool), class_ids

# Charger et préparer le dataset pour l'entraînement et la validation
dataset_dir = 'data_t'
train_dataset = FlowerDataset()
train_dataset.load_flowers(dataset_dir, "train")
train_dataset.prepare()

val_dataset = FlowerDataset()
val_dataset.load_flowers(dataset_dir, "val")
val_dataset.prepare()

# Initialiser le modèle Mask R-CNN
model = MaskRCNN(mode="training", config=FlowerConfig(), model_dir='./logs3_epoche40_final')

# Charger les poids du modèle pré-entraîné sur COCO, en excluant les couches spécifiques aux classes
model.load_weights(
    'mask_rcnn_coco.h5', 
    by_name=True, 
    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]
)

# Entraîner le modèle avec augmentation des données
try:
    augmentation = imgaug.augmenters.Fliplr(0.5)  # Appliquer une symétrie horizontale aléatoire à 50%
    model.train(
        train_dataset,  # Jeu de données d'entraînement
        val_dataset,  # Jeu de données de validation
        learning_rate=FlowerConfig().LEARNING_RATE,  # Taux d'apprentissage
        epochs=40,  # Nombre d'époques
        layers='heads',  # Entraîner uniquement les couches "heads" (heads layers)
        augmentation=augmentation  # Ajouter l'augmentation des données
    )
except Exception as e:
    # Enregistrer l'erreur dans un fichier log et afficher un message
    logging.error(f"Erreur pendant l'entraînement : {str(e)}")
    print(f"Une erreur est survenue pendant l'entraînement, mais l'exécution continue : {e}")
